The goal of this notebook is to create a geodaframe containing information about each solar panel annotation. This dataframe will be useful in creating image mask pairs of tiles of the Cape Town geotiff images. The code to create the geodataframe and the image make pairs is available in this notebook.   

In [3]:
# import necessary packages

# Import JSON library for working with JSON data
import json
# Import pandas for data manipulation and analysis
import pandas as pd
# Import geopandas for working with geospatial data
import geopandas as gpd
# Import os for interacting with the operating system (file paths, directories, etc.)
import os
# Import numpy for numerical operations
import numpy as np
# Import rasterio for reading and writing raster data (e.g., GeoTIFFs)
import rasterio
# Import ast for parsing strings into Python objects (like lists, dicts)
import ast
# Import OpenCV for computer vision tasks (image processing)
import cv2
# Import re for regular expressions (string pattern matching)
import re
# Import shutil for high-level file operations (copying, moving, removing files/directories)
import shutil
# Import random for generating random numbers and selections
import random
# Import rowcol and Window from rasterio for working with pixel positions and windowed reads
from rasterio.transform import rowcol
from rasterio.windows import Window
# Import Polygon, shape, and box from shapely for geometric operations (like polygons and bounding boxes)
from shapely.geometry import Polygon
from shapely.geometry import shape
from shapely.geometry import box
# Import namedtuple from collections for creating lightweight object types
from collections import namedtuple
# Import tqdm for progress bars (especially useful in loops)
from tqdm import tqdm
# Import Image module from PIL for opening, manipulating, and saving images
from PIL import Image

In [4]:
# Select the images that have been annotated
status = pd.read_excel(
    '/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Aerial Imagery/CapeTown_ImageIDs.xlsx',
    engine='openpyxl'
)
completed = status[status["Status"]=='Completed']
completed = completed[['Image ID', 'Annotator']]
completed

/home/cmn60/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Image ID,Annotator
56,W07A_1,Gary Alvarez Mejia
63,W07A_16,Abby Finkle
72,W07A_24,Vanshika Mittal
106,W07C_10,Biz Yoder
107,W07C_11,Biz Yoder
...,...,...
2720,W59C_10,Abby Finkle
2737,W59D_20,Abby Finkle
2761,W66D_10,Abby Finkle
2776,W66D_4,Abby Finkle


There are 2807 images of Cape Town in total. 
As of Nov 29, there 544 completed images, including those that have no solar panel annotation. 151 geotiff images have annotations.
As of Fev 1st, there 731 completed images, including those that have no solar panel annotation. 251 geotiff images have annotations.

In [5]:
completed['Annotator'].unique()

array(['Gary Alvarez Mejia', 'Abby Finkle', 'Vanshika Mittal',
       'Biz Yoder', 'Zeinab Mukhtar', 'Shehr Naz Ashraf', 'Ye Khaung Oo',
       'Veena Shirsath', 'Brian Mulu Mutua', 'Halle Evans',
       'Ummamah Shah', 'Fiona Bolte-Bradhurst'], dtype=object)

Abby Finkle has only one completed image. Gary Alvarez Mejia has two.

I couldn't find Abby Finkle's annotations layer. Veena didn't upload an annotations layer.

In [6]:
# Filter completed annotations
selected_annotators = ['Gary Alvarez Mejia',  'Vanshika Mittal',
       'Biz Yoder', 'Zeinab Mukhtar', 'Shehr Naz Ashraf', 'Ye Khaung Oo',
        'Brian Mulu Mutua', 'Halle Evans',
       'Ummamah Shah', 'Fiona Bolte-Bradhurst']
filtered_completed_df = completed[completed['Annotator'].isin(selected_annotators)]

# Map annotator names to keys
key_mapping = {
    'Biz Yoder': 'biz',
    'Brian Mulu Mutua': 'mutua',
    'Fiona Bolte-Bradhurst': 'fiona',
    'Ummamah Shah': 'shah',
    'Ye Khaung Oo': 'ye',
    'Gary Alvarez Mejia': 'mejia',
    'Vanshika Mittal': 'mittal',
    'Zeinab Mukhtar': 'mukhtar',
    'Shehr Naz Ashraf': 'shehr',
    'Halle Evans': 'evans'
}

# Set file paths
file_paths = {
    'biz': r"/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Aerial Imagery/Annotation layers/Biz Yoder/yoder_annotations 0128_v2.shp",
    
    'mutua': r"/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Aerial Imagery/Annotation layers/Brian Mulu Mutua/mutua_annotations 1027.shp",
    'fiona': r"/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Aerial Imagery/Annotation layers/Fiona Bolte-Bradhurst/1.31/bolte.bradhurst_annotations_1.31.shp",
    'shah': r"/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Aerial Imagery/Annotation layers/Ummamah Shah/Shah_6M.shp",
    'ye': r"/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Aerial Imagery/Annotation layers/Ye Khaung Oo/final_2020_annotations.shp",
    'mejia': r"/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Aerial Imagery/Annotation layers/Gary Alvarez Mejia/11-13/alvarezmejia_annotations.shp",
    'mittal': r"/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Aerial Imagery/Annotation layers/Vanshika Mittal/Mittal_annotations.shp",
    'mukhtar': r"/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Aerial Imagery/Annotation layers/Zeinab Mukhtar/mukhtar_annotations.shp",
    'shehr': r"/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Aerial Imagery/Annotation layers/Shehr Naz Ashraf/ashraf_annotations.shp",
    'evans': r"/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Aerial Imagery/Annotation layers/Halle Evans/evans_annotations_layer.shp"
}


annotator_image_ids = filtered_completed_df.groupby('Annotator')['Image ID'].apply(list).to_dict()
annotator_image_ids = {key_mapping[old_key]: value for old_key, value in annotator_image_ids.items()}

In [7]:
# Extract the CRS of the images. This will be used to reproject the annotations layers to this CRS to avoid any inconsistencies.
input_tif = r"/home/il72/cape_town_annotation_checker/db_pipeline/download/images/2023_RGB_8cm_W18B_5.tif"
with rasterio.open(input_tif) as dataset:
    geotif_crs = dataset.crs

# Clean the annotation layers be removing those marked as PV pool and PV heater
def clean_annotation_layers(layer):
    copy = layer
    copy = copy.to_crs(geotif_crs)
    copy['area'] = copy.geometry.area
    
    # remove annotations tagged as PV pool and PV heater
    if 'PV_Pool' in copy.columns:
        copy = copy[copy['PV_Pool'] != 1]

    if 'PV_pool' in copy.columns:
        copy = copy[copy['PV_pool'] != 1]

    if 'PV_heater' in copy.columns:
        copy = copy[copy['PV_heater'] != 1]
    
    columns_to_drop = ['layer', 'path', 'PV_heater', 'uncertflag', 'PV_Pool', 'PV_pool']
    copy = copy.drop([col for col in columns_to_drop if col in copy.columns], axis=1) 
    
    copy.reset_index(drop=True, inplace=True)       
    
    return copy

In [8]:
# Load and process annotations
annotations_list = []
for annotator, path in file_paths.items():
    annotations = gpd.read_file(path)
    # print(annotations.columns)
    # print(annotator)
    annotations['annotator'] = annotator
    if (annotator != 'biz') & (annotator != 'evans'):
        annotations = annotations[annotations['path'].isnull()]
    annotations = clean_annotation_layers(annotations)
    annotations_list.append(annotations)

# Concatenate all annotations and add important polygon features (polygon's centroid coordinates)
annotations = pd.concat(annotations_list, ignore_index=True)
annotations.reset_index(drop=True, inplace=True)
annotations['id'] = annotations.index
annotations['centroid'] = annotations.geometry.centroid
annotations['centroid_latitude'] = annotations.centroid.y
annotations['centroid_longitude'] = annotations.centroid.x
annotations.drop(columns=['centroid'], inplace=True)
annotations = annotations[annotations['geometry'].notnull()]

# Add additional columns to annotations
annotations[['image_name', 'nw_corner_of_image_latitude', 'nw_corner_of_image_longitude', 
             'se_corner_of_image_latitude', 'se_corner_of_image_longitude']] = None

annotations

,id,area,geometry,annotator,centroid_latitude,centroid_longitude,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude
0,0,23.162657,"POLYGON ((-19993.55 -3769759.6, -19988.801 -37...",biz,-3.769763e+06,-19991.996891,None,None,None,None,None
1,1,8.567160,"POLYGON ((-13352.818 -3768899.929, -13345.695 ...",biz,-3.768902e+06,-13349.578806,None,None,None,None,None
2,2,9.287182,"POLYGON ((-13353.655 -3768901.389, -13346.512 ...",biz,-3.768904e+06,-13350.422909,None,None,None,None,None
3,3,8.580228,"POLYGON ((-13354.608 -3768903.024, -13347.583 ...",biz,-3.768906e+06,-13351.395241,None,None,None,None,None
4,4,10.808848,"POLYGON ((-13354.102 -3768907.616, -13353.441 ...",biz,-3.768910e+06,-13351.308980,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
20987,20987,7967.342250,"POLYGON ((-32416.279 -3760650.449, -32378.19 -...",evans,-3.760649e+06,-32342.015986,None,None,None,None,None
20988,20988,2453.418159,"POLYGON ((-32259.265 -3760454.46, -32266.355 -...",evans,-3.760454e+06,-32221.597883,None,None,None,None,None
20989,20989,1.662698,"POLYGON ((-32200.421 -3760240.994, -32199.184 ...",evans,-3.760242e+06,-32199.980018,None,None,None,None,None
20990,20990,4.017408,"POLYGON ((-32122.904 -3760859.814, -32121.646 ...",evans,-3.760861e+06,-32122.286606,None,None,None,None,None


Next, we need to determine which image each polygon belongs to. We are doing this by calculating the border coordinates for each annotator's images, then checking which polygons are within the images' bounds.

In [9]:
def get_image_border_coordinates(image_path):
    try:
        with rasterio.open(image_path) as src:
            return src.bounds
    except rasterio.errors.RasterioIOError as e:
        print(f"Warning: Failed to read {image_path} — {e}")
        return None

folder_path = '/home/il72/cape_town_annotation_checker/db_pipeline/download/images'
prefix = '2023_RGB_8cm_'

annotator_border_coordinates = {}
completed_ann = []
processed_images = {}

total_files = 0
skipped_files = 0

for annotator, image_names in annotator_image_ids.items():
    border_coordinates = {}
    for image_name in image_names:
        total_files += 1
        full_name = f"{prefix}{image_name}.tif"
        image_path = os.path.join(folder_path, full_name)

        if os.path.exists(image_path):
            if image_name not in processed_images:
                coordinates = get_image_border_coordinates(image_path)
                if coordinates:
                    processed_images[image_name] = coordinates
                else:
                    skipped_files += 1
                    continue
            else:
                coordinates = processed_images[image_name]

            border_coordinates[image_name] = coordinates
        else:
            print(f"File not found: {image_path}")
            skipped_files += 1

    annotator_border_coordinates[annotator] = border_coordinates
    completed_ann.append(annotator)

print(f"\nTotal files attempted: {total_files}")
print(f"Files skipped (due to missing or errors): {skipped_files}")
print(f"Successfully processed files: {total_files - skipped_files}")


Total files attempted: 245
Files skipped (due to missing or errors): 0
Successfully processed files: 245


In [10]:
# some more data manipulations before associating the polygons to the images
BoundingBox = namedtuple('BoundingBox', ['left', 'bottom', 'right', 'top'])

transformed_dict = {}
for annotator, images in annotator_border_coordinates.items():
    transformed_dict[annotator] = {image: {'left': bounds.left, 'bottom': bounds.bottom, 'right': bounds.right, 'top': bounds.top} for image, bounds in images.items()}

annotator_border_coordinates = transformed_dict 

# Function to check if the centroid (consequently the polygon) is within a bounding box
def is_point_within_bounds(lat, lon, bounds):
    return bounds['left'] <= lon <= bounds['right'] and bounds['bottom'] <= lat <= bounds['top']

In [11]:
flattened_coordinates = {}
for annotator, images in annotator_border_coordinates.items():
    flattened_coordinates.update(images)

# Iterate over each row in the annotations DataFrame
for idx, row in annotations.iterrows():
    # annotator = row['annotator']
    centroid_lat = row['centroid_latitude']
    centroid_lon = row['centroid_longitude']
    
    # Check which image the centroid belongs to
    for image_name, bounds in flattened_coordinates.items():
        if is_point_within_bounds(centroid_lat, centroid_lon, bounds):
            annotations.loc[idx, 'image_name'] = image_name
            annotations.loc[idx, 'nw_corner_of_image_latitude'] = bounds['top']
            annotations.loc[idx, 'nw_corner_of_image_longitude'] = bounds['left']
            annotations.loc[idx, 'se_corner_of_image_latitude'] = bounds['bottom']
            annotations.loc[idx, 'se_corner_of_image_longitude'] = bounds['right']
            break

# Now we associated the polygons to the images
annotations

,id,area,geometry,annotator,centroid_latitude,centroid_longitude,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude
0,0,23.162657,"POLYGON ((-19993.55 -3769759.6, -19988.801 -37...",biz,-3.769763e+06,-19991.996891,W16C_21,-3769000.0,-20000.0,-3770000.0,-19000.0
1,1,8.567160,"POLYGON ((-13352.818 -3768899.929, -13345.695 ...",biz,-3.768902e+06,-13349.578806,W16D_17,-3768000.0,-14000.0,-3769000.0,-13000.0
2,2,9.287182,"POLYGON ((-13353.655 -3768901.389, -13346.512 ...",biz,-3.768904e+06,-13350.422909,W16D_17,-3768000.0,-14000.0,-3769000.0,-13000.0
3,3,8.580228,"POLYGON ((-13354.608 -3768903.024, -13347.583 ...",biz,-3.768906e+06,-13351.395241,W16D_17,-3768000.0,-14000.0,-3769000.0,-13000.0
4,4,10.808848,"POLYGON ((-13354.102 -3768907.616, -13353.441 ...",biz,-3.768910e+06,-13351.308980,W16D_17,-3768000.0,-14000.0,-3769000.0,-13000.0
...,...,...,...,...,...,...,...,...,...,...,...
20987,20987,7967.342250,"POLYGON ((-32416.279 -3760650.449, -32378.19 -...",evans,-3.760649e+06,-32342.015986,None,None,None,None,None
20988,20988,2453.418159,"POLYGON ((-32259.265 -3760454.46, -32266.355 -...",evans,-3.760454e+06,-32221.597883,None,None,None,None,None
20989,20989,1.662698,"POLYGON ((-32200.421 -3760240.994, -32199.184 ...",evans,-3.760242e+06,-32199.980018,None,None,None,None,None
20990,20990,4.017408,"POLYGON ((-32122.904 -3760859.814, -32121.646 ...",evans,-3.760861e+06,-32122.286606,None,None,None,None,None


In [17]:
# Saving the processed dataframe locally
# available_annotations = annotations[annotations['image_name'].notnull()]
# available_annotations.to_file(r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Teams\Team 1 Machine learning\CT - MachineLearning\S1 Machine Learning\available_annotations_feb.shp")

So far, we created a dataset that contains all of the available annotations, their geographical coordinates, each polygon centroid's coordinates, the annotator name, and the image each annotation belongs to. Now, we will calculate the pixel coordinates and the area in pixels of each polygon. These pixel corrdinates are necessary to cut the images into image-mask pairs.

In [12]:
# This cell will run for a while. It calculates the pixel metrics of each polygon.
annotations = annotations[annotations['image_name'].notnull()]

def calculate_pixel_metrics_grouped(group, image_base_path):
    metrics = []
    for _, row in group.iterrows():
        geotiff_path = image_base_path + row['image_name'] + ".tif"
        try:
            with rasterio.open(geotiff_path) as src:
                transform = src.transform
                metric = calculate_pixel_metrics_dynamic(row, transform)
                metrics.append(metric)
        except FileNotFoundError:
            print(f"GeoTIFF not found: {geotiff_path}")
            metrics.append(None)
    
    return pd.DataFrame(metrics, index=group.index)

def calculate_pixel_metrics_dynamic(row, transform):
    polygon = shape(row['geometry'])
    centroid = polygon.centroid

    centroid_pixel_row, centroid_pixel_col = rowcol(transform, centroid.x, centroid.y)

    pixel_vertices = [
        rowcol(transform, vertex[0], vertex[1]) for vertex in polygon.exterior.coords
    ]

    pixel_polygon = shape({
        'type': 'Polygon',
        'coordinates': [[(c, r) for r, c in pixel_vertices]]
    })
    pixel_area = pixel_polygon.area

    return {
        'polygon_vertices_pixels': pixel_vertices,
        'centroid_latitude_pixels': centroid_pixel_row,
        'centroid_longitude_pixels': centroid_pixel_col,
        'area_pixels': pixel_area
    }

# Group the annotations by image_name
grouped_annotations = annotations.groupby('image_name')

image_base_path = r'/home/il72/cape_town_annotation_checker/db_pipeline/download/images/2023_RGB_8cm_'
pixel_metrics_list = []

for name, group in tqdm(grouped_annotations, desc="Processing Images"):
    print(f"Processing image: {name}")
    metrics = calculate_pixel_metrics_grouped(group, image_base_path)
    pixel_metrics_list.append(metrics)

# Concatenate all metrics into a single DataFrame
pixel_metrics_df = pd.concat(pixel_metrics_list)

# Assign each metric to its respective column in the annotations DataFrame
annotations = annotations.join(pixel_metrics_df)

Processing Images:   0%|          | 0/206 [00:00<?, ?it/s]

Processing image: W07A_1


Processing Images:   0%|          | 1/206 [00:00<03:04,  1.11it/s]

Processing image: W07A_24
Processing image: W07C_10


Processing Images:   1%|▏         | 3/206 [00:01<01:09,  2.91it/s]

...

Processing Images:  99%|█████████▉| 204/206 [01:00<00:00,  2.72it/s]

Processing image: W57C_8


Processing Images: 100%|█████████▉| 205/206 [01:01<00:00,  3.01it/s]

Processing image: W57C_9


Processing Images: 100%|██████████| 206/206 [01:01<00:00,  3.34it/s]


In [19]:
with rasterio.open('/home/cmn60/cape_town_segmentation/data/Oxnard/618051897.tif') as src:
    image = src.read()
    

In [13]:
# Check the order of the columns of the annotations dataframe and replace them here
annotations.columns = ['id', 'annotator', 'area', 'centroid_latitude', 'centroid_longitude',
       'image_name', 'nw_corner_of_image_latitude',
       'nw_corner_of_image_longitude', 'se_corner_of_image_latitude',
       'se_corner_of_image_longitude', 'polygon_vertices_pixels',
       'centroid_latitude_pixels', 'centroid_longitude_pixels', 'area_pixels',
       'geometry']

# Save the final dataframe. Now we are ready to create the image-mask pairs
annotations.to_file(r'/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Teams/S1 Machine Learning/annotations_final_feb_updated.geojson')

Some necessary data manipulation of the annotations dataframe's polygon_vertices_pixels column before creating image mask pairs.

In [14]:

df = annotations.copy()

# Apply cleaning only if value is a string
def clean_polygon_string(x):
    if isinstance(x, str):
        # Remove np.int32()
        x = re.sub(r'np\.int32\((\d+)\)', r'\1', x)
        # Replace (x, y) with [x, y]
        x = re.sub(r'\((\d+),\s*(\d+)\)', r'[\1, \2]', x)
        # Fix incomplete tuples/lists
        tuples = re.findall(r'\[\d+, \d+\]', x)
        return '[' + ', '.join(tuples) + ']'
    return x  # Leave as-is (e.g. already parsed)

df['polygon_vertices_pixels'] = df['polygon_vertices_pixels'].apply(clean_polygon_string)

# Now safely parse strings into Python lists
def safe_literal_eval(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception as e:
            print(f"Failed to parse: {x}\nError: {e}")
            return None
    return x  # Already a list or not a string

df['polygon_vertices_pixels'] = df['polygon_vertices_pixels'].apply(safe_literal_eval)

print(df['polygon_vertices_pixels'])


0       -19000.0
1       -13000.0
2       -13000.0
3       -13000.0
4       -13000.0
          ...   
20948   -32000.0
20949   -31000.0
20950   -31000.0
20951   -31000.0
20952   -31000.0
Name: polygon_vertices_pixels, Length: 12293, dtype: float64


In [15]:
annotations = df

Now that the dataframe is ready, we can finally create the image mask pairs. Below are the functions we use to do so

In [16]:
def create_mask(image_shape, polygons):
    mask = np.zeros(image_shape[:2], dtype="uint8")
    for polygon in polygons:
        cv2.fillPoly(mask, [polygon], 255)
    flipped_mask = cv2.flip(mask, 0)
    rotated_mask = cv2.rotate(flipped_mask, cv2.ROTATE_90_CLOCKWISE)
    return rotated_mask

def save_tile_and_mask(tile, mask, tile_index_pixels, tile_dir, mask_dir, image_name):
    tile_filename = os.path.join(tile_dir, f'i_{image_name}_{tile_index_pixels}.png')
    mask_filename = os.path.join(mask_dir, f'm_{image_name}_{tile_index_pixels}.png')
    cv2.imwrite(tile_filename, cv2.cvtColor(tile, cv2.COLOR_RGB2BGR))
    cv2.imwrite(mask_filename, mask)

def adjust_polygon_coordinates(polygons, x_offset, y_offset):
    adjusted_polygons = []
    for polygon in polygons:
        adjusted_polygon = polygon - np.array([x_offset, y_offset])
        adjusted_polygons.append(adjusted_polygon)
    return adjusted_polygons

def process_geotiff(image_name, geotiff_path, tile_size, df, tile_dir, mask_dir):
    with rasterio.open(geotiff_path) as src:
        geotiff_array = src.read()

        if len(geotiff_array.shape) == 3:
            geotiff_array = np.transpose(geotiff_array, (1, 2, 0))

        height, width = geotiff_array.shape[:2]

        # Calculate padding
        pad_height = (tile_size - height % tile_size) % tile_size
        pad_width = (tile_size - width % tile_size) % tile_size

        # Add padding to the image
        padded_image = np.pad(geotiff_array, ((0, pad_height), (0, pad_width), (0, 0)), mode='constant')

        padded_height, padded_width = padded_image.shape[:2]

        tile_index = 0
        for y in range(0, padded_height, tile_size):
            for x in range(0, padded_width, tile_size):
                tile = padded_image[y:y+tile_size, x:x+tile_size]

                polygons_in_tile = []
                for _, row in df.iterrows():
                    polygon = row['polygon_vertices_pixels']
                    
                    bounds = {
                        'left': x,
                        'right': x+tile_size,
                        'bottom': y+tile_size,
                        'top': y
                    }
                    
                    if (bounds['left'] <= row['centroid_longitude_pixels'] <= bounds['right'] and bounds['top'] <= row['centroid_latitude_pixels'] <= bounds['bottom']):
                        polygons_in_tile.append(polygon)
                    
                adjusted_polygons = adjust_polygon_coordinates(polygons_in_tile, y, x)

                mask = create_mask(tile.shape, adjusted_polygons)
                
                # save only the masks and tiles that contain annotations
                if np.any(mask > 0):
                    tile_index_pixels = str(int(y/tile_size)) + "_" + str(int(x/tile_size))
                    save_tile_and_mask(tile, mask, tile_index_pixels, tile_dir, mask_dir, image_name)
                
                tile_index += 1


def process_all_images_in_folder(folder_path, annotations_df, tile_size, tile_dir, mask_dir, processed_images_list):
    unique_images = annotations_df['image_name'].unique()
    idx = 0
    for image_name in unique_images:
        image_path = os.path.join(folder_path, f"{image_name}.tif")

        
        if os.path.exists(image_path):
            print(image_name)
            print(idx)
            idx += 1
            processed_images_list.append(image_name)
            
            image_annotations_df = annotations_df[annotations_df['image_name'] == image_name]
            process_geotiff(image_name, image_path, tile_size, image_annotations_df, tile_dir, mask_dir)


In [24]:
tile_dir = r'/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Teams/S1 Machine Learning/dataset/tiles_320'
mask_dir = r'/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Teams/S1 Machine Learning/dataset/masks_320'
os.makedirs(tile_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# path where the original images are saved
dataset_path = r"/home/il72/cape_town_annotation_checker/db_pipeline/download/images/2023_RGB_8cm_"

# Choose the tile size of your new dataset
tile_size = 320 

# The names of the processed images will be printed to monitor the progress
processed_images_list = []
process_all_images_in_folder(dataset_path, annotations, tile_size=320, tile_dir=tile_dir, mask_dir=mask_dir, processed_images_list=processed_images_list)

There are a lot of images that don't contain any solar panel. To avoid wasting training resources, we'll only select the images that contain the panels to work with (it may still be useful to train the model with the images that don't contain any panels to increase model robustness)

In [25]:
# Organizing the folders locally -> copy the masks and tiles to the destination folders
# In the following, we are only keeping the masks that have annotations. In the future, it may be better to train the models with all masks, even empty ones

# copy the masks that contain the target to a seperate folder
def check_mask_has_target(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    return np.any(mask > 0)

def copy_masks_with_target(source_folder, destination_folder):
    os.makedirs(destination_folder, exist_ok=True)
    
    for filename in os.listdir(source_folder):
        file_path = os.path.join(source_folder, filename)
        
        if check_mask_has_target(file_path):
            shutil.copy(file_path, destination_folder)
            print(f"Copied {filename} to {destination_folder}")
            
# copy the corresponding images to a new folder
def copy_corresponding_images(mask_folder, image_folder, destination_folder):
    os.makedirs(destination_folder, exist_ok=True)
    
    for mask_filename in os.listdir(mask_folder):
        image_filename = 'i' + mask_filename[1:]
        image_path = os.path.join(image_folder, image_filename)
        
        if os.path.exists(image_path):
            shutil.copy(image_path, destination_folder)
            print(f"Copied {image_filename} to {destination_folder}")



Organize the data folders

In [ ]:
source_folder = '/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Teams/S1 Machine Learning/dataset/masks_320'
destination_folder = '/home/cmn60/cape_town_segmentation/data/cape_town/masks_target'

copy_masks_with_target(source_folder, destination_folder)

mask_folder = '/home/cmn60/cape_town_segmentation/data/cape_town/masks_target'
image_folder = '/home/cmn60/cape_town_segmentation/data/cape_town/tiles'
destination_folder = '/home/cmn60/cape_town_segmentation/data/cape_town/images_target'

copy_corresponding_images(mask_folder, image_folder, destination_folder)

Copied m_W48C_11_14_11.png to /home/cmn60/cape_town_segmentation/data/cape_town/masks_target
Copied m_W57B_18_6_37.png to /home/cmn60/cape_town_segmentation/data/cape_town/masks_target
Copied m_W36A_19_29_34.png to /home/cmn60/cape_town_segmentation/data/cape_town/masks_target
Copied m_W25B_16_3_8.png to /home/cmn60/cape_town_segmentation/data/cape_town/masks_target
Copied m_W25A_7_27_5.png to /home/cmn60/cape_town_segmentation/data/cape_town/masks_target
Copied m_W16C_19_17_26.png to /home/cmn60/cape_town_segmentation/data/cape_town/masks_target
Copied m_W36B_10_18_32.png to /home/cmn60/cape_town_segmentation/data/cape_town/masks_target
Copied m_W53B_24_23_24.png to /home/cmn60/cape_town_segmentation/data/cape_town/masks_target
Copied m_W25A_3_36_14.png to /home/cmn60/cape_town_segmentation/data/cape_town/masks_target
Copied m_W16C_19_28_13.png to /home/cmn60/cape_town_segmentation/data/cape_town/masks_target
Copied m_W57B_8_11_0.png to /home/cmn60/cape_town_segmentation/data/cape_tow

In [28]:
image_folder = '/home/cmn60/cape_town_segmentation/Cape Town Energy Transitions Bass Connections/Class Materials 2024-2025/Teams/S1 Machine Learning/dataset/tiles_320'
copy_corresponding_images(mask_folder, image_folder, destination_folder)

Copied i_W48C_11_14_11.png to /home/cmn60/cape_town_segmentation/data/cape_town/images_target
Copied i_W57B_18_6_37.png to /home/cmn60/cape_town_segmentation/data/cape_town/images_target
Copied i_W36A_19_29_34.png to /home/cmn60/cape_town_segmentation/data/cape_town/images_target
Copied i_W25B_16_3_8.png to /home/cmn60/cape_town_segmentation/data/cape_town/images_target
Copied i_W25A_7_27_5.png to /home/cmn60/cape_town_segmentation/data/cape_town/images_target
Copied i_W16C_19_17_26.png to /home/cmn60/cape_town_segmentation/data/cape_town/images_target
Copied i_W36B_10_18_32.png to /home/cmn60/cape_town_segmentation/data/cape_town/images_target
Copied i_W53B_24_23_24.png to /home/cmn60/cape_town_segmentation/data/cape_town/images_target
Copied i_W25A_3_36_14.png to /home/cmn60/cape_town_segmentation/data/cape_town/images_target
Copied i_W16C_19_28_13.png to /home/cmn60/cape_town_segmentation/data/cape_town/images_target
Copied i_W57B_8_11_0.png to /home/cmn60/cape_town_segmentation/dat

In [29]:
# split the dataset into train, test, and val datasets. You can split the folders from the terminal

def split_data(mask_folder, image_folder, train_folder, val_folder, test_folder, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    os.makedirs(os.path.join(train_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(train_folder, 'masks'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'masks'), exist_ok=True)
    os.makedirs(os.path.join(test_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(test_folder, 'masks'), exist_ok=True)
    
    mask_files = os.listdir(mask_folder)
    
    random.shuffle(mask_files)
    
    total_files = len(mask_files)
    train_count = int(total_files * train_ratio)
    val_count = int(total_files * val_ratio)
    test_count = total_files - train_count - val_count
    
    train_files = mask_files[:train_count]
    val_files = mask_files[train_count:train_count + val_count]
    test_files = mask_files[train_count + val_count:]
    
    def copy_files(file_list, dest_image_folder, dest_mask_folder):
        for mask_filename in file_list:
            shutil.copy(os.path.join(mask_folder, mask_filename), dest_mask_folder)
            
            image_filename = 'i' + mask_filename[1:]
            shutil.copy(os.path.join(image_folder, image_filename), dest_image_folder)
    
    copy_files(train_files, os.path.join(train_folder, 'images'), os.path.join(train_folder, 'masks'))
    copy_files(val_files, os.path.join(val_folder, 'images'), os.path.join(val_folder, 'masks'))
    copy_files(test_files, os.path.join(test_folder, 'images'), os.path.join(test_folder, 'masks'))

mask_folder = '/home/cmn60/cape_town_segmentation/data/cape_town/masks_target'
image_folder = '/home/cmn60/cape_town_segmentation/data/cape_town/images_target'
train_folder = '/home/cmn60/cape_town_segmentation/data/cape_town/train'
val_folder = '/home/cmn60/cape_town_segmentation/data/cape_town/val'
test_folder = '/home/cmn60/cape_town_segmentation/data/cape_town/test'

split_data(mask_folder, image_folder, train_folder, val_folder, test_folder)

In [27]:
import torch


print(torch.__version__)


2.1.0+cu121


In [1]:
import torch

ckpt_path = "/data/users/cmn60/cape_town_segmentation/trained_models/cal_320_new/last.ckpt"
ckpt = torch.load(ckpt_path, map_location="cpu")

# See top-level keys
print(ckpt.keys())

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'MixedPrecision'])


In [2]:
print(ckpt['epoch'])  # The epoch number when this was saved

# If ModelCheckpoint tracked metrics
if 'callbacks' in ckpt:
    for cb in ckpt['callbacks']:
        print(cb)

99
ModelCheckpoint{'monitor': 'valid_dataset_iou', 'mode': 'min', 'every_n_train_steps': 0, 'every_n_epochs': 1, 'train_time_interval': None}
